In [ ]:
#require "core";;
#require "yaml.unix";;
#require "yaml";;
#require "fpath"
#require "yaml.unix"
#require "lwt"
#require "cohttp"
#require "cohttp-lwt-unix"
#use "/home/opam/work/lib/util.ml"

In [4]:
open Lwt
open Cohttp
open Cohttp_lwt_unix

let (alpaca_secret, alpaca_key) = 
  let yaml = Yaml_unix.of_file_exn (Fpath.v "keys.paper.yaml") in
  let key = yaml |> get_value "key" |> get_string_opt |> get_string in 
  let secret = yaml |> get_value "secret" |> get_string_opt |> get_string in
  (secret, key)
  
let headers = Header.init ()
  |> fun h -> Header.add_list h 
              [("APCA-API-KEY-ID", alpaca_key); 
               ("APCA-API-SECRET-KEY", alpaca_secret)]
            

let paca_time =
  let uri = Uri.of_string "https://paper-api.alpaca.markets/v2/clock" in
    Cohttp_lwt_unix.Client.get ~headers  uri >>= fun (resp, body) ->
      let code = resp |> Response.status |> Code.code_of_status in
      Printf.printf "Response code: %d\n" code;
      Printf.printf "Headers: %s\n" (resp |> Response.headers |> Header.to_string);
      body |> Cohttp_lwt.Body.to_string >|= fun body ->
      Printf.printf "Body of length: %d\n" (String.length body);
      body

let () =
  let respBody = Lwt_main.run paca_time in
  print_endline (respBody)

val alpaca_secret : string = "qEmsGIbj8WSBK3DLVDlwcB6KJ3JsLcndnuCHdLIu"
val alpaca_key : string = "PKLDSENB0U8ST5KZI612"


val headers : Cohttp.Header.t = <abstr>


val paca_time : string Lwt.t = <abstr>


Response code: 200
Headers: access-control-allow-credentials: true
connection: keep-alive
content-length: 147
content-type: application/json; charset=UTF-8
date: Fri, 05 Feb 2021 15:10:13 GMT
server: nginx/1.16.1
vary: Origin


Body of length: 147
{"timestamp":"2021-02-05T10:10:13.382865659-05:00","is_open":true,"next_open":"2021-02-08T09:30:00-05:00","next_close":"2021-02-05T16:00:00-05:00"}
